# Python3 + Appium + Andriod 趣头条脚本

功能包括，签到，金币领取，新闻阅读，小视频，视频

## 开发环境配置

[Selenium 安装](https://germey.gitbooks.io/python3webspider/content/1.2.2-Selenium%E7%9A%84%E5%AE%89%E8%A3%85.html)
[appnium 安装](https://germey.gitbooks.io/python3webspider/content/1.7.3-Appium%E7%9A%84%E5%AE%89%E8%A3%85.html)


### Appium基础

- platformName: 手机操作系统
- deviceName: 手机类型
将andriod手机链接电脑后，电脑打开Appium，同时手机打开USB调试功能，可通过 `adb devices -l` 命令来测试链接状态，如果出现类似结果，说明PC已经正确连接手机。

```
List of devices attached
568bcabc               device product:clover model:MI_PAD_4 device:clover transport_id:3
```

- newCommandTimeout: 两条appium命令间的最长时间间隔，若超过这个时间，appium会自动结束并退出app

- noReset, fullReset: noReset 不要在会话前重置应用状态。默认值false。 fullReset (Android) 通过卸载而不是清空数据来重置应用状态。在Android上, 这也会在会话结束后自动清除被测应用。默认值false。【建议用 noReset, 可以保存设置中的登录状态，这样滤过复杂的登录验证】


- appActivity, appPackage: appActivity与appPackage指用于启动待测app的activityName与packageName

In [ ]:
from appium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time,random,datetime
from appium.webdriver.common.touch_action import TouchAction
from appium.webdriver.common.mobileby import MobileBy

PLATFORM = 'Android'
DEVICE_NAME = 'MI_PAD_4'
DRIVER_SERVER = 'http://localhost:4723/wd/hub'
TIMEOUT = 600
NORESET = 'True'
FLICK_STRAT_X = 300
FLICK_START_Y = 300
FLICK_DISTANCE = 700

class Automation():
    
    def __init__(self,APP_PACKAGE,APP_ACTIVITY):
        self.desired_caps = {
            'platformName':PLATFORM,
            'deviceName': DEVICE_NAME,
            'appPackage': APP_PACKAGE,
            'appActivity': APP_ACTIVITY,
            'noReset': NORESET,
            'newCommandTimeout': TIMEOUT,
        }
        print('打开 appium 服务器...')
        print('配置 appium ...')
        self.driver = webdriver.Remote(DRIVER_SERVER,self.desired_caps)
        self.wait = WebDriverWait(self.driver,30)
        self.size = self.driver.get_window_size()
        self.x = x = self.size['width']*0.5
        self.y1 = self.size['height']*random.uniform(0.75,0.99)
        self.y2 = self.size['height']*random.uniform(0.01,0.25)        
        self.time =  random.uniform(800,1200)   
        
        
    def swipeUp(self):
        '''向上滑动屏幕'''
        self.driver.swipe(self.x,self.y1,self.x,self.y2,self.time)
        print('向上滑动屏幕')
    
    
    def swipeDown(self):
        '''向下滑动屏幕'''
        self.driver.swipe(self.x,self.y2,self.x,self.y1,self.time)      
        print('向下滑动屏幕')
        
        
    def control_task(self,func,t):
        starttime = datetime.datetime.now()
        while True:
            func()
            endtime = datetime.datetime.now()
            runtime = (endtime-starttime).seconds
            if runtime > t:
                break
        
        
class Qutoutiao(Automation):
    
    def get_coins(self):
        
        
#         try:
#             pop_up = self.wait.until(EC.presence_of_element_located((By.ID, 'com.jifen.qukan:id/rv')))
#             #pop_up.click()
#             print('取消推送通知...')
#         except:
#             print('无推送通知弹出框...')          
        
#         try:
#             pop_up = self.wait.until(EC.presence_of_element_located((By.ID, 'com.jifen.qukan:id/rv')))
#             #pop_up.click()
#             print('取消签到提醒...')
#         except:
#             print('无签到提醒弹出框...')   
#         try:
#             lock_popup = self.wait.until(EC.presence_of_element_located((By.ID, 'com.jifen.qukan:id/ry')))
#             lock_popup.click()
#             print('取消锁屏提醒...')
#         except:
#             print('无锁屏提醒弹出框...')           
        
        coins = self.wait.until(EC.presence_of_element_located((By.ID, 'com.jifen.qukan.title:id/pf')))
        if coins.get_attribute('text') == '领取':
            coins.click()
            print('领取金币')
        else:
            print('无金币可领取')
        
    def mission(self):
        mission = self.wait.until(EC.presence_of_element_located((By.ID, 'com.jifen.qukan:id/k0')))
        mission.click()
        print('点击底部菜单:任务...')         
        print('签到...')
    
    def video(self,interval):
        
        video = self.wait.until(EC.presence_of_element_located((By.ID, 'com.jifen.qukan:id/jx')))
        video.click()
        print('点击底部菜单:视频...')
        t = 1
        start = time.time()
        while True:
            content = self.driver.find_elements_by_id("com.jifen.qukan:id/a57")
            time_text = self.driver.find_elements_by_id("com.jifen.qukan:id/a6w")
            print('第{}次进入循环...'.format(t))
            for i in range(len(content)):
                try:
                    time_span = time_text[i].get_attribute('text').split(':')
                    content[i].click()
                    print('点击第{}视频'.format(i+1))
                    time_cost = int(time_span[0])*60+int(time_span[1])
                    time.sleep(random.randint(4,10))
                    print('该视频时长:{}s'.format(time_cost))
                    time.sleep(time_cost) 
                except IndexError:
                    print('此为广告...')
                    continue
            self.driver.find_element_by_id("com.jifen.qukan:id/jx").click()
            print('点击刷新页面...')
            t += 1
            end  = start + interval
            now = time.time()
            if now >= end:
                print('结束视频循环')
                break  
            
    def smallvideo(self,interval):
        smallvideo = self.wait.until(EC.presence_of_element_located((By.ID, 'com.jifen.qukan:id/jy')))
        smallvideo.click()
        print('点击底部菜单:小视频...') 
        start = time.time()
        while True:
            time.sleep(random.randint(5,20))
            self.swipeUp()
            end = start + interval
            now = time.time()
            if now >= end:
                print('结束小视频循环')
                break  
    
            
    def read_article(self,interval):
        sz_menu = self.wait.until(EC.presence_of_element_located((By.XPATH, '/hierarchy/android.widget.FrameLayout/android.widget.LinearLayout/android.widget.FrameLayout/android.widget.LinearLayout/android.widget.FrameLayout/android.widget.RelativeLayout/android.widget.FrameLayout[2]/android.widget.LinearLayout/android.widget.RelativeLayout/android.widget.RelativeLayout/android.widget.HorizontalScrollView/android.widget.LinearLayout/android.widget.TextView[3]')))
        sz_menu.click()
        print('点击顶部菜单:深圳...')
        start = time.time()
        while True:
            articles = self.wait.until(EC.presence_of_all_elements_located((By.ID, 'com.jifen.qukan:id/a35')))
            print('新闻数量',len(articles))
            for article in articles:
                print('进入新闻{}'.format(article.get_attribute('text')))
                article.click()
                for i in range(random.randint(2,6)):
                    time.sleep(random.randint(6,10))
                    self.swipeUp()
                for i in range(random.randint(2,6)):
                    time.sleep(random.randint(6,10))
                    self.swipeDown()  
                try:
                    return_button = self.wait.until(EC.presence_of_element_located((By.ID, 'com.jifen.qukan:id/fi')))
                except:
                    try:
                        return_button = self.wait.until(EC.presence_of_element_located((By.ID, 'com.jifen.qukan:id/mn')))
                    except:
                        return_button = self.wait.until(EC.presence_of_element_located((By.ID, 'com.jifen.qukan:id/i5')))
                finally:
                    return_button.click()
                print('退出新闻')
            for i in range(2):
                time.sleep(random.randint(1,3))
                self.swipeUp()        
            end = start + interval
            now = time.time()
            if now >= end:
                print('结束新闻循环')
                break  
           
        
    def main(self):
        self.get_coins()
        self.mission()
        self.read_article(600)
        self.smallvideo(600)
        self.video(600)
        self.smallvideo(600)
        self.video(600)
        

qutoutiao_appPackage = 'com.jifen.qukan'
qutoutiao_appActivity = 'com.jifen.qkbase.main.MainActivity'

if __name__ == '__main__':
    qutoutiao = Qutoutiao(qutoutiao_appPackage,qutoutiao_appActivity)
    qutoutiao.main()
